# Transaction Utility 3 

This notebook aims to recreate some of the empirical findings of Thaler, R. (1985). Mental accounting and consumer choice. Marketing Science, 4(3), 199-214. 
Specifically we are interested in whether LLMs' responses are similar to the original responses in **section 3** of the paper, regarding the Hockey Ticket Question.
However, this time, we will change the numbers mentioned in the prompts. (Anchoring heuristics)

---

In order to run this notebook an openAI API key, as well as Replicate API token are required.  
Further they have to be set as environment variables named OPENAI_API_KEY and REPLICATE_API_TOKEN respectively.

Throughout the process of this project, adjustments have been made to this notebook, therefore the original outputs are no longer included.   
For test purposes, the notebook was re-run with a very low number of iterations per prompt and every export of prompts, dictionaries or results was commented out.

---

In [1]:
from openai import OpenAI
import openai
import matplotlib.pyplot as plt
import os 
import numpy as np
import pandas as pd
from tqdm import tqdm
import replicate
from ast import literal_eval
import plotly.graph_objects as go
import pickle
from collections import Counter

In [2]:
# Get openAI API key (previously saved as environmental variable)
openai.api_key = os.environ["OPENAI_API_KEY"]

# Set client
client = OpenAI()

# Set global plot style
plt.style.use('seaborn-v0_8')

# Set plots to be displayed in notebook
%matplotlib inline

---

#### Setting up prompts for the experiment



- LLMs used in the experiment:
    - GPT-3.5-Turbo         (ID = 1)
    - GPT-4-1106-Preview    (ID = 2)
    - LLama-70b             (ID = 3)

We can differentiate between the following scenario combinations:

- Actual ticket price:
    - $5 * pi                (ID = 1)
    - $5 * 10                (ID = 2)

- Initial costs:
    - free                  (ID = 1)
    - actual price          (ID = 2)
    - double actual price   (ID = 3)
- Current market price:
    - actual price                (ID = 1)
    - double actual price         (ID = 2)
- Selling to:
    - Friend                (ID = 1)
    - Stranger              (ID = 2)



Similar to the Prospect Theory and Decoy Effect notebooks, we will use experiment IDs to run the study. The IDs will be constructed as:

*TU_model__actualprice_initialcosts_currentprice_buyer*

Therefore, TU3_2_2_2_1_2 would mean we used GPT-4-1106-Preview, an actual ticket price of $50, initial costs of $50, a current market price of $50 as well and we are selling to a stranger.

- Constructing the prompts

In [3]:
# Original price from study
original_price = 5
actual_price = np.round(original_price * np.pi,2) # to give odd number 

# Set up list of initial costs
initial_costs = ["but you were given your tickets for free by a friend.", "which is what you paid for each ticket.", f"but you paid ${actual_price * 2} each for your tickets when you bought them."]

# Set up list of current ticket prices
orientation_prices = [f"${actual_price}", f"${actual_price * 2}"]

# Set up list of potential buyers in a scenario
potential_buyers = ["friend?", "stranger?"]

TU3_prompts_pi = []
for costs in initial_costs:
    for orientation_price in orientation_prices:
        for potential_buyer in potential_buyers:
            prompt = f"""Imagine that you are going to a soldout Cornell hockey playoff game, and you have an extra ticket to sell or give away. The price marked on the ticket is ${actual_price} {costs}
            You get to the game early to make sure you get rid of the ticket. An informal survey of people selling tickets indicates that the going price is {orientation_price}. 
            You find someone who wants the ticket and takes out his wallet to pay you. He asks, how much you want for the ticket. 
            Assume that there is no law against charging a price higher than that marked on the ticket. What price do you ask for, if he is a {potential_buyer}"""
            TU3_prompts_pi.append(prompt)

In [4]:
# Original price from study
original_price = 5
actual_price = original_price * 10 

# Set up list of initial costs
initial_costs = ["but you were given your tickets for free by a friend.", "which is what you paid for each ticket.", f"but you paid ${actual_price * 2} each for your tickets when you bought them."]

# Set up list of current ticket prices
orientation_prices = [f"{actual_price}", f"{actual_price * 2}"]

# Set up list of potential buyers in a scenario
potential_buyers = ["friend?", "stranger?"]

TU3_prompts_10 = []
for costs in initial_costs:
    for orientation_price in orientation_prices:
        for potential_buyer in potential_buyers:
            prompt = f"""Imagine that you are going to a soldout Cornell hockey playoff game, and you have an extra ticket to sell or give away. The price marked on the ticket is ${actual_price} {costs}
            You get to the game early to make sure you get rid of the ticket. An informal survey of people selling tickets indicates that the going price is ${orientation_price}. 
            You find someone who wants the ticket and takes out his wallet to pay you. He asks, how much you want for the ticket. 
            Assume that there is no law against charging a price higher than that marked on the ticket. What price do you ask for, if he is a {potential_buyer}"""
            TU3_prompts_10.append(prompt)

In [5]:
# Save prompts for use in Dashboard
TU3_prompts = TU3_prompts_pi + TU3_prompts_10
#with open('Dashboard/src/data/Input/TU3_prompts.pkl', 'wb') as f:
#    pickle.dump(TU3_prompts, f)

- TU3_prompts[0]: 5 x pi, free, 5 x pi, friend -> **_1_1_1_1** (Configuration 1)
- TU3_prompts[1]: 5 x pi, free, 5 x pi, stranger -> **_1_1_1_2** (Configuration 2)
- TU3_prompts[2]: 5 x pi, free, 5 x pi x 2, friend -> **_1_1_2_1** (Configuration 3)
- TU3_prompts[3]: 5 x pi, free, 5 x pi x 2, stranger -> **_1_1_2_2** (Configuration 4)
- TU3_prompts[4]: 5 x pi, 5 x pi, 5 x pi, friend -> **_1_2_1_1** (Configuration 5) 
- TU3_prompts[5]: 5 x pi, 5 x pi, 5 x pi, stranger -> **_1_2_1_2** (Configuration 6)
- TU3_prompts[6]: 5 x pi, 5 x pi, 5 x pi x 2, friend ->**_1_2_2_1** (Configuration 7)
- TU3_prompts[7]: 5 x pi, 5 x pi, 5 x pi x 2, stranger ->**_1_2_2_2** (Configuration 8)
- TU3_prompts[8]: 5 x pi, 5 x pi x 2, 5 x pi, friend ->**_1_3_1_1** (Configuration 9)
- TU3_prompts[9]: 5 x pi, 5 x pi x 2, 5 x pi, stranger ->**_1_3_1_2** (Configuration 10)
- TU3_prompts[10]: 5 x pi, 5 x pi x 2, 5 x pi x 2x friend -> **_1_3_2_1** (Configuration 11)
- TU3_prompts[11]: 5 x pi, 5 x pi x 2, 5 x pi x 2x stranger -> **_1_3_2_2** (Configuration 12)
- TU3_prompts[12]: 50, free, 50, friend -> **_2_1_1_1** (Configuration 13)
- TU3_prompts[13]: 50, free, 50, stranger -> **_2_1_1_2** (Configuration 14)
- TU3_prompts[14]: 50, free, 100, friend -> **_2_1_2_1** (Configuration 15)
- TU3_prompts[15]: 50, free, 100, stranger -> **_2_1_2_2**(Configuration 16)
- TU_prompts[16]: 50, 50, 50, friend -> **_2_2_1_1** (Configuration 17)
- TU_prompts[17]: 50, 50, 50, stranger -> **_2_2_1_2** (Configuration 18)
- TU_prompts[18]: 50, 50, 100, friend -> **_2_2_2_1** (Configuration 19)
- TU_prompts[19]: 50, 50, 100, stranger -> **_2_2_2_2** (Configuration 20)
- TU_prompts[20]: 50, 100, 50, friend -> **_2_3_1_1** (Configuration 21)
- TU_prompts[21]: 50, 100, 50, stranger -> **_2_3_1_2** (Configuration 22)
- TU_prompts[22]: 50, 100, 100, friend -> **_2_3_2_1**(Configuration 23)
- TU_prompts[23]: 50, 100, 100, stranger -> **_2_3_2_2** (Configuration 24)

---

#### Dictionaries to extract information about the different experiments

In [6]:
# Dictionary to look up prompt for a given experiment id. key: experiment id, value: prompt
TU3_experiment_prompts_dict = {
    "TU3_1_1_1_1_1": TU3_prompts[0],
    "TU3_1_1_1_1_2": TU3_prompts[1],
    "TU3_1_1_1_2_1": TU3_prompts[2],
    "TU3_1_1_1_2_2": TU3_prompts[3],
    "TU3_1_1_2_1_1": TU3_prompts[4],
    "TU3_1_1_2_1_2": TU3_prompts[5],
    "TU3_1_1_2_2_1": TU3_prompts[6],
    "TU3_1_1_2_2_2": TU3_prompts[7],
    "TU3_1_1_3_1_1": TU3_prompts[8],
    "TU3_1_1_3_1_2": TU3_prompts[9],
    "TU3_1_1_3_2_1": TU3_prompts[10],
    "TU3_1_1_3_2_2": TU3_prompts[11],
    "TU3_1_2_1_1_1": TU3_prompts[12],
    "TU3_1_2_1_1_2": TU3_prompts[13],
    "TU3_1_2_1_2_1": TU3_prompts[14],
    "TU3_1_2_1_2_2": TU3_prompts[15],
    "TU3_1_2_2_1_1": TU3_prompts[16],
    "TU3_1_2_2_1_2": TU3_prompts[17],
    "TU3_1_2_2_2_1": TU3_prompts[18],
    "TU3_1_2_2_2_2": TU3_prompts[19],
    "TU3_1_2_3_1_1": TU3_prompts[20],
    "TU3_1_2_3_1_2": TU3_prompts[21],
    "TU3_1_2_3_2_1": TU3_prompts[22],
    "TU3_1_2_3_2_2": TU3_prompts[23],
    "TU3_2_1_1_1_1": TU3_prompts[0],
    "TU3_2_1_1_1_2": TU3_prompts[1],
    "TU3_2_1_1_2_1": TU3_prompts[2],
    "TU3_2_1_1_2_2": TU3_prompts[3],
    "TU3_2_1_2_1_1": TU3_prompts[4],
    "TU3_2_1_2_1_2": TU3_prompts[5],
    "TU3_2_1_2_2_1": TU3_prompts[6],
    "TU3_2_1_2_2_2": TU3_prompts[7],
    "TU3_2_1_3_1_1": TU3_prompts[8],
    "TU3_2_1_3_1_2": TU3_prompts[9],
    "TU3_2_1_3_2_1": TU3_prompts[10],
    "TU3_2_1_3_2_2": TU3_prompts[11],
    "TU3_2_2_1_1_1": TU3_prompts[12],
    "TU3_2_2_1_1_2": TU3_prompts[13],
    "TU3_2_2_1_2_1": TU3_prompts[14],
    "TU3_2_2_1_2_2": TU3_prompts[15],
    "TU3_2_2_2_1_1": TU3_prompts[16],
    "TU3_2_2_2_1_2": TU3_prompts[17],
    "TU3_2_2_2_2_1": TU3_prompts[18],
    "TU3_2_2_2_2_2": TU3_prompts[19],
    "TU3_2_2_3_1_1": TU3_prompts[20],
    "TU3_2_2_3_1_2": TU3_prompts[21],
    "TU3_2_2_3_2_1": TU3_prompts[22],
    "TU3_2_2_3_2_2": TU3_prompts[23],
    "TU3_3_1_1_1_1": TU3_prompts[0],
    "TU3_3_1_1_1_2": TU3_prompts[1],
    "TU3_3_1_1_2_1": TU3_prompts[2],
    "TU3_3_1_1_2_2": TU3_prompts[3],
    "TU3_3_1_2_1_1": TU3_prompts[4],
    "TU3_3_1_2_1_2": TU3_prompts[5],
    "TU3_3_1_2_2_1": TU3_prompts[6],
    "TU3_3_1_2_2_2": TU3_prompts[7],
    "TU3_3_1_3_1_1": TU3_prompts[8],
    "TU3_3_1_3_1_2": TU3_prompts[9],
    "TU3_3_1_3_2_1": TU3_prompts[10],
    "TU3_3_1_3_2_2": TU3_prompts[11],
    "TU3_3_2_1_1_1": TU3_prompts[12],
    "TU3_3_2_1_1_2": TU3_prompts[13],
    "TU3_3_2_1_2_1": TU3_prompts[14],
    "TU3_3_2_1_2_2": TU3_prompts[15],
    "TU3_3_2_2_1_1": TU3_prompts[16],
    "TU3_3_2_2_1_2": TU3_prompts[17],
    "TU3_3_2_2_2_1": TU3_prompts[18],
    "TU3_3_2_2_2_2": TU3_prompts[19],
    "TU3_3_2_3_1_1": TU3_prompts[20],
    "TU3_3_2_3_1_2": TU3_prompts[21],
    "TU3_3_2_3_2_1": TU3_prompts[22],
    "TU3_3_2_3_2_2": TU3_prompts[23],    
}

# Dictionary to look up model for a given experiment id. key: experiment id, value: model
TU3_model_dict = {
    "TU3_1_1_1_1_1": "gpt-3.5-turbo",
    "TU3_1_1_1_1_2": "gpt-3.5-turbo",
    "TU3_1_1_1_2_1": "gpt-3.5-turbo",
    "TU3_1_1_1_2_2": "gpt-3.5-turbo",
    "TU3_1_1_2_1_1": "gpt-3.5-turbo",
    "TU3_1_1_2_1_2": "gpt-3.5-turbo",
    "TU3_1_1_2_2_1": "gpt-3.5-turbo",
    "TU3_1_1_2_2_2": "gpt-3.5-turbo",
    "TU3_1_1_3_1_1": "gpt-3.5-turbo",
    "TU3_1_1_3_1_2": "gpt-3.5-turbo",
    "TU3_1_1_3_2_1": "gpt-3.5-turbo",
    "TU3_1_1_3_2_2": "gpt-3.5-turbo",
    "TU3_1_2_1_1_1": "gpt-3.5-turbo",
    "TU3_1_2_1_1_2": "gpt-3.5-turbo",
    "TU3_1_2_1_2_1": "gpt-3.5-turbo",
    "TU3_1_2_1_2_2": "gpt-3.5-turbo",
    "TU3_1_2_2_1_1": "gpt-3.5-turbo",
    "TU3_1_2_2_1_2": "gpt-3.5-turbo",
    "TU3_1_2_2_2_1": "gpt-3.5-turbo",
    "TU3_1_2_2_2_2": "gpt-3.5-turbo",
    "TU3_1_2_3_1_1": "gpt-3.5-turbo",
    "TU3_1_2_3_1_2": "gpt-3.5-turbo",
    "TU3_1_2_3_2_1": "gpt-3.5-turbo",
    "TU3_1_2_3_2_2": "gpt-3.5-turbo",
    "TU3_2_1_1_1_1": "gpt-4-1106-preview",
    "TU3_2_1_1_1_2": "gpt-4-1106-preview",
    "TU3_2_1_1_2_1": "gpt-4-1106-preview",
    "TU3_2_1_1_2_2": "gpt-4-1106-preview",
    "TU3_2_1_2_1_1": "gpt-4-1106-preview",
    "TU3_2_1_2_1_2": "gpt-4-1106-preview",
    "TU3_2_1_2_2_1": "gpt-4-1106-preview",
    "TU3_2_1_2_2_2": "gpt-4-1106-preview",
    "TU3_2_1_3_1_1": "gpt-4-1106-preview",
    "TU3_2_1_3_1_2": "gpt-4-1106-preview",
    "TU3_2_1_3_2_1": "gpt-4-1106-preview",
    "TU3_2_1_3_2_2": "gpt-4-1106-preview",
    "TU3_2_2_1_1_1": "gpt-4-1106-preview",
    "TU3_2_2_1_1_2": "gpt-4-1106-preview",
    "TU3_2_2_1_2_1": "gpt-4-1106-preview",
    "TU3_2_2_1_2_2": "gpt-4-1106-preview",
    "TU3_2_2_2_1_1": "gpt-4-1106-preview",
    "TU3_2_2_2_1_2": "gpt-4-1106-preview",
    "TU3_2_2_2_2_1": "gpt-4-1106-preview",
    "TU3_2_2_2_2_2": "gpt-4-1106-preview",
    "TU3_2_2_3_1_1": "gpt-4-1106-preview",
    "TU3_2_2_3_1_2": "gpt-4-1106-preview",
    "TU3_2_2_3_2_1": "gpt-4-1106-preview",
    "TU3_2_2_3_2_2": "gpt-4-1106-preview",
    "TU3_3_1_1_1_1": "meta/llama-2-70b-chat:02e509c789964a7ea8736978a43525956ef40397be9033abf9fd2badfe68c9e3",
    "TU3_3_1_1_1_2": "meta/llama-2-70b-chat:02e509c789964a7ea8736978a43525956ef40397be9033abf9fd2badfe68c9e3",
    "TU3_3_1_1_2_1": "meta/llama-2-70b-chat:02e509c789964a7ea8736978a43525956ef40397be9033abf9fd2badfe68c9e3",
    "TU3_3_1_1_2_2": "meta/llama-2-70b-chat:02e509c789964a7ea8736978a43525956ef40397be9033abf9fd2badfe68c9e3",
    "TU3_3_1_2_1_1": "meta/llama-2-70b-chat:02e509c789964a7ea8736978a43525956ef40397be9033abf9fd2badfe68c9e3",
    "TU3_3_1_2_1_2": "meta/llama-2-70b-chat:02e509c789964a7ea8736978a43525956ef40397be9033abf9fd2badfe68c9e3",
    "TU3_3_1_2_2_1": "meta/llama-2-70b-chat:02e509c789964a7ea8736978a43525956ef40397be9033abf9fd2badfe68c9e3",
    "TU3_3_1_2_2_2": "meta/llama-2-70b-chat:02e509c789964a7ea8736978a43525956ef40397be9033abf9fd2badfe68c9e3",
    "TU3_3_1_3_1_1": "meta/llama-2-70b-chat:02e509c789964a7ea8736978a43525956ef40397be9033abf9fd2badfe68c9e3",
    "TU3_3_1_3_1_2": "meta/llama-2-70b-chat:02e509c789964a7ea8736978a43525956ef40397be9033abf9fd2badfe68c9e3",
    "TU3_3_1_3_2_1": "meta/llama-2-70b-chat:02e509c789964a7ea8736978a43525956ef40397be9033abf9fd2badfe68c9e3",
    "TU3_3_1_3_2_2": "meta/llama-2-70b-chat:02e509c789964a7ea8736978a43525956ef40397be9033abf9fd2badfe68c9e3",
    "TU3_3_2_1_1_1": "meta/llama-2-70b-chat:02e509c789964a7ea8736978a43525956ef40397be9033abf9fd2badfe68c9e3",
    "TU3_3_2_1_1_2": "meta/llama-2-70b-chat:02e509c789964a7ea8736978a43525956ef40397be9033abf9fd2badfe68c9e3",
    "TU3_3_2_1_2_1": "meta/llama-2-70b-chat:02e509c789964a7ea8736978a43525956ef40397be9033abf9fd2badfe68c9e3",
    "TU3_3_2_1_2_2": "meta/llama-2-70b-chat:02e509c789964a7ea8736978a43525956ef40397be9033abf9fd2badfe68c9e3",
    "TU3_3_2_2_1_1": "meta/llama-2-70b-chat:02e509c789964a7ea8736978a43525956ef40397be9033abf9fd2badfe68c9e3",
    "TU3_3_2_2_1_2": "meta/llama-2-70b-chat:02e509c789964a7ea8736978a43525956ef40397be9033abf9fd2badfe68c9e3",
    "TU3_3_2_2_2_1": "meta/llama-2-70b-chat:02e509c789964a7ea8736978a43525956ef40397be9033abf9fd2badfe68c9e3",
    "TU3_3_2_2_2_2": "meta/llama-2-70b-chat:02e509c789964a7ea8736978a43525956ef40397be9033abf9fd2badfe68c9e3",
    "TU3_3_2_3_1_1": "meta/llama-2-70b-chat:02e509c789964a7ea8736978a43525956ef40397be9033abf9fd2badfe68c9e3",
    "TU3_3_2_3_1_2": "meta/llama-2-70b-chat:02e509c789964a7ea8736978a43525956ef40397be9033abf9fd2badfe68c9e3",
    "TU3_3_2_3_2_1": "meta/llama-2-70b-chat:02e509c789964a7ea8736978a43525956ef40397be9033abf9fd2badfe68c9e3",
    "TU3_3_2_3_2_2": "meta/llama-2-70b-chat:02e509c789964a7ea8736978a43525956ef40397be9033abf9fd2badfe68c9e3",    
}

# Dictionary to look up prompt id for a given experiment id. key: experiment id, value: prompt id
TU3_prompt_ids_dict = {
    "TU3_1_1_1_1_1": "TU3_prompts[0]",
    "TU3_1_1_1_1_2": "TU3_prompts[1]",
    "TU3_1_1_1_2_1": "TU3_prompts[2]",
    "TU3_1_1_1_2_2": "TU3_prompts[3]",
    "TU3_1_1_2_1_1": "TU3_prompts[4]",
    "TU3_1_1_2_1_2": "TU3_prompts[5]",
    "TU3_1_1_2_2_1": "TU3_prompts[6]",
    "TU3_1_1_2_2_2": "TU3_prompts[7]",
    "TU3_1_1_3_1_1": "TU3_prompts[8]",
    "TU3_1_1_3_1_2": "TU3_prompts[9]",
    "TU3_1_1_3_2_1": "TU3_prompts[10]",
    "TU3_1_1_3_2_2": "TU3_prompts[11]",
    "TU3_1_2_1_1_1": "TU3_prompts[12]",
    "TU3_1_2_1_1_2": "TU3_prompts[13]",
    "TU3_1_2_1_2_1": "TU3_prompts[14]",
    "TU3_1_2_1_2_2": "TU3_prompts[15]",
    "TU3_1_2_2_1_1": "TU3_prompts[16]",
    "TU3_1_2_2_1_2": "TU3_prompts[17]",
    "TU3_1_2_2_2_1": "TU3_prompts[18]",
    "TU3_1_2_2_2_2": "TU3_prompts[19]",
    "TU3_1_2_3_1_1": "TU3_prompts[20]",
    "TU3_1_2_3_1_2": "TU3_prompts[21]",
    "TU3_1_2_3_2_1": "TU3_prompts[22]",
    "TU3_1_2_3_2_2": "TU3_prompts[23]",
    "TU3_2_1_1_1_1": "TU3_prompts[0]",
    "TU3_2_1_1_1_2": "TU3_prompts[1]",
    "TU3_2_1_1_2_1": "TU3_prompts[2]",
    "TU3_2_1_1_2_2": "TU3_prompts[3]",
    "TU3_2_1_2_1_1": "TU3_prompts[4]",
    "TU3_2_1_2_1_2": "TU3_prompts[5]",
    "TU3_2_1_2_2_1": "TU3_prompts[6]",
    "TU3_2_1_2_2_2": "TU3_prompts[7]",
    "TU3_2_1_3_1_1": "TU3_prompts[8]",
    "TU3_2_1_3_1_2": "TU3_prompts[9]",
    "TU3_2_1_3_2_1": "TU3_prompts[10]",
    "TU3_2_1_3_2_2": "TU3_prompts[11]",
    "TU3_2_2_1_1_1": "TU3_prompts[12]",
    "TU3_2_2_1_1_2": "TU3_prompts[13]",
    "TU3_2_2_1_2_1": "TU3_prompts[14]",
    "TU3_2_2_1_2_2": "TU3_prompts[15]",
    "TU3_2_2_2_1_1": "TU3_prompts[16]",
    "TU3_2_2_2_1_2": "TU3_prompts[17]",
    "TU3_2_2_2_2_1": "TU3_prompts[18]",
    "TU3_2_2_2_2_2": "TU3_prompts[19]",
    "TU3_2_2_3_1_1": "TU3_prompts[20]",
    "TU3_2_2_3_1_2": "TU3_prompts[21]",
    "TU3_2_2_3_2_1": "TU3_prompts[22]",
    "TU3_2_2_3_2_2": "TU3_prompts[23]",
    "TU3_3_1_1_1_1": "TU3_prompts[0]",
    "TU3_3_1_1_1_2": "TU3_prompts[1]",
    "TU3_3_1_1_2_1": "TU3_prompts[2]",
    "TU3_3_1_1_2_2": "TU3_prompts[3]",
    "TU3_3_1_2_1_1": "TU3_prompts[4]",
    "TU3_3_1_2_1_2": "TU3_prompts[5]",
    "TU3_3_1_2_2_1": "TU3_prompts[6]",
    "TU3_3_1_2_2_2": "TU3_prompts[7]",
    "TU3_3_1_3_1_1": "TU3_prompts[8]",
    "TU3_3_1_3_1_2": "TU3_prompts[9]",
    "TU3_3_1_3_2_1": "TU3_prompts[10]",
    "TU3_3_1_3_2_2": "TU3_prompts[11]",
    "TU3_3_2_1_1_1": "TU3_prompts[12]",
    "TU3_3_2_1_1_2": "TU3_prompts[13]",
    "TU3_3_2_1_2_1": "TU3_prompts[14]",
    "TU3_3_2_1_2_2": "TU3_prompts[15]",
    "TU3_3_2_2_1_1": "TU3_prompts[16]",
    "TU3_3_2_2_1_2": "TU3_prompts[17]",
    "TU3_3_2_2_2_1": "TU3_prompts[18]",
    "TU3_3_2_2_2_2": "TU3_prompts[19]",
    "TU3_3_2_3_1_1": "TU3_prompts[20]",
    "TU3_3_2_3_1_2": "TU3_prompts[21]",
    "TU3_3_2_3_2_1": "TU3_prompts[22]",
    "TU3_3_2_3_2_2": "TU3_prompts[23]",    
}

# Dictionary to look up actual priced marked on ticket for a given experiment id. key: experiment id, value: actual price
TU3_actual_price_dict = {
    "TU3_1_1_1_1_1": 15.71,
    "TU3_1_1_1_1_2": 15.71,
    "TU3_1_1_1_2_1": 15.71,
    "TU3_1_1_1_2_2": 15.71,
    "TU3_1_1_2_1_1": 15.71,
    "TU3_1_1_2_1_2": 15.71,
    "TU3_1_1_2_2_1": 15.71,
    "TU3_1_1_2_2_2": 15.71,
    "TU3_1_1_3_1_1": 15.71,
    "TU3_1_1_3_1_2": 15.71,
    "TU3_1_1_3_2_1": 15.71,
    "TU3_1_1_3_2_2": 15.71,
    "TU3_1_2_1_1_1": 50,
    "TU3_1_2_1_1_2": 50,
    "TU3_1_2_1_2_1": 50,
    "TU3_1_2_1_2_2": 50,
    "TU3_1_2_2_1_1": 50,
    "TU3_1_2_2_1_2": 50,
    "TU3_1_2_2_2_1": 50,
    "TU3_1_2_2_2_2": 50,
    "TU3_1_2_3_1_1": 50,
    "TU3_1_2_3_1_2": 50,
    "TU3_1_2_3_2_1": 50,
    "TU3_1_2_3_2_2": 50,
    "TU3_2_1_1_1_1": 15.71,
    "TU3_2_1_1_1_2": 15.71,
    "TU3_2_1_1_2_1": 15.71,
    "TU3_2_1_1_2_2": 15.71,
    "TU3_2_1_2_1_1": 15.71,
    "TU3_2_1_2_1_2": 15.71,
    "TU3_2_1_2_2_1": 15.71,
    "TU3_2_1_2_2_2": 15.71,
    "TU3_2_1_3_1_1": 15.71,
    "TU3_2_1_3_1_2": 15.71,
    "TU3_2_1_3_2_1": 15.71,
    "TU3_2_1_3_2_2": 15.71,
    "TU3_2_2_1_1_1": 50,
    "TU3_2_2_1_1_2": 50,
    "TU3_2_2_1_2_1": 50,
    "TU3_2_2_1_2_2": 50,
    "TU3_2_2_2_1_1": 50,
    "TU3_2_2_2_1_2": 50,
    "TU3_2_2_2_2_1": 50,
    "TU3_2_2_2_2_2": 50,
    "TU3_2_2_3_1_1": 50,
    "TU3_2_2_3_1_2": 50,
    "TU3_2_2_3_2_1": 50,
    "TU3_2_2_3_2_2": 50,
    "TU3_3_1_1_1_1": 15.71,
    "TU3_3_1_1_1_2": 15.71,
    "TU3_3_1_1_2_1": 15.71,
    "TU3_3_1_1_2_2": 15.71,
    "TU3_3_1_2_1_1": 15.71,
    "TU3_3_1_2_1_2": 15.71,
    "TU3_3_1_2_2_1": 15.71,
    "TU3_3_1_2_2_2": 15.71,
    "TU3_3_1_3_1_1": 15.71,
    "TU3_3_1_3_1_2": 15.71,
    "TU3_3_1_3_2_1": 15.71,
    "TU3_3_1_3_2_2": 15.71,
    "TU3_3_2_1_1_1": 50,
    "TU3_3_2_1_1_2": 50,
    "TU3_3_2_1_2_1": 50,
    "TU3_3_2_1_2_2": 50,
    "TU3_3_2_2_1_1": 50,
    "TU3_3_2_2_1_2": 50,
    "TU3_3_2_2_2_1": 50,
    "TU3_3_2_2_2_2": 50,
    "TU3_3_2_3_1_1": 50,
    "TU3_3_2_3_1_2": 50,
    "TU3_3_2_3_2_1": 50,
    "TU3_3_2_3_2_2": 50,    
}

# Dictionary to look up price paid for the ticket: key: experiment id, value: price paid
TU3_initial_costs_dict = {
    "TU3_1_1_1_1_1": 0,
    "TU3_1_1_1_1_2": 0,
    "TU3_1_1_1_2_1": 0,
    "TU3_1_1_1_2_2": 0,
    "TU3_1_1_2_1_1": 15.71,
    "TU3_1_1_2_1_2": 15.71,
    "TU3_1_1_2_2_1": 15.71,
    "TU3_1_1_2_2_2": 15.71,
    "TU3_1_1_3_1_1": 31.42,
    "TU3_1_1_3_1_2": 31.42,
    "TU3_1_1_3_2_1": 31.42,
    "TU3_1_1_3_2_2": 31.42,
    "TU3_1_2_1_1_1": 0,
    "TU3_1_2_1_1_2": 0,
    "TU3_1_2_1_2_1": 0,
    "TU3_1_2_1_2_2": 0,
    "TU3_1_2_2_1_1": 50,
    "TU3_1_2_2_1_2": 50,
    "TU3_1_2_2_2_1": 50,
    "TU3_1_2_2_2_2": 50,
    "TU3_1_2_3_1_1": 100,
    "TU3_1_2_3_1_2": 100,
    "TU3_1_2_3_2_1": 100,
    "TU3_1_2_3_2_2": 100,
    "TU3_2_1_1_1_1": 0,
    "TU3_2_1_1_1_2": 0,
    "TU3_2_1_1_2_1": 0,
    "TU3_2_1_1_2_2": 0,
    "TU3_2_1_2_1_1": 15.71,
    "TU3_2_1_2_1_2": 15.71,
    "TU3_2_1_2_2_1": 15.71,
    "TU3_2_1_2_2_2": 15.71,
    "TU3_2_1_3_1_1": 31.42,
    "TU3_2_1_3_1_2": 31.42,
    "TU3_2_1_3_2_1": 31.42,
    "TU3_2_1_3_2_2": 31.42,
    "TU3_2_2_1_1_1": 0,
    "TU3_2_2_1_1_2": 0,
    "TU3_2_2_1_2_1": 0,
    "TU3_2_2_1_2_2": 0,
    "TU3_2_2_2_1_1": 50,
    "TU3_2_2_2_1_2": 50,
    "TU3_2_2_2_2_1": 50,
    "TU3_2_2_2_2_2": 50,
    "TU3_2_2_3_1_1": 100,
    "TU3_2_2_3_1_2": 100,
    "TU3_2_2_3_2_1": 100,
    "TU3_2_2_3_2_2": 100,
    "TU3_3_1_1_1_1": 0,
    "TU3_3_1_1_1_2": 0,
    "TU3_3_1_1_2_1": 0,
    "TU3_3_1_1_2_2": 0,
    "TU3_3_1_2_1_1": 15.71,
    "TU3_3_1_2_1_2": 15.71,
    "TU3_3_1_2_2_1": 15.71,
    "TU3_3_1_2_2_2": 15.71,
    "TU3_3_1_3_1_1": 31.42,
    "TU3_3_1_3_1_2": 31.42,
    "TU3_3_1_3_2_1": 31.42,
    "TU3_3_1_3_2_2": 31.42,
    "TU3_3_2_1_1_1": 0,
    "TU3_3_2_1_1_2": 0,
    "TU3_3_2_1_2_1": 0,
    "TU3_3_2_1_2_2": 0,
    "TU3_3_2_2_1_1": 50,
    "TU3_3_2_2_1_2": 50,
    "TU3_3_2_2_2_1": 50,
    "TU3_3_2_2_2_2": 50,
    "TU3_3_2_3_1_1": 100,
    "TU3_3_2_3_1_2": 100,
    "TU3_3_2_3_2_1": 100,
    "TU3_3_2_3_2_2": 100,   
}  

# Dictionary to look up current market price for the ticket. key: experiment id, value: current market price
TU3_orientation_price_dict = {
    "TU3_1_1_1_1_1": 15.71,
    "TU3_1_1_1_1_2": 15.71,
    "TU3_1_1_1_2_1": 31.42,
    "TU3_1_1_1_2_2": 31.42,
    "TU3_1_1_2_1_1": 15.71,
    "TU3_1_1_2_1_2": 15.71,
    "TU3_1_1_2_2_1": 31.42,
    "TU3_1_1_2_2_2": 31.42,
    "TU3_1_1_3_1_1": 15.71,
    "TU3_1_1_3_1_2": 15.71,
    "TU3_1_1_3_2_1": 31.42,
    "TU3_1_1_3_2_2": 31.42,
    "TU3_1_2_1_1_1": 50,
    "TU3_1_2_1_1_2": 50,
    "TU3_1_2_1_2_1": 100,
    "TU3_1_2_1_2_2": 100,
    "TU3_1_2_2_1_1": 50,
    "TU3_1_2_2_1_2": 50,
    "TU3_1_2_2_2_1": 100,
    "TU3_1_2_2_2_2": 100,
    "TU3_1_2_3_1_1": 50,
    "TU3_1_2_3_1_2": 50,
    "TU3_1_2_3_2_1": 100,
    "TU3_1_2_3_2_2": 100,
    "TU3_2_1_1_1_1": 15.71,
    "TU3_2_1_1_1_2": 15.71,
    "TU3_2_1_1_2_1": 31.42,
    "TU3_2_1_1_2_2": 31.42,
    "TU3_2_1_2_1_1": 15.71,
    "TU3_2_1_2_1_2": 15.71,
    "TU3_2_1_2_2_1": 31.42,
    "TU3_2_1_2_2_2": 31.42,
    "TU3_2_1_3_1_1": 15.71,
    "TU3_2_1_3_1_2": 15.71,
    "TU3_2_1_3_2_1": 31.42,
    "TU3_2_1_3_2_2": 31.42,
    "TU3_2_2_1_1_1": 50,
    "TU3_2_2_1_1_2": 50,
    "TU3_2_2_1_2_1": 100,
    "TU3_2_2_1_2_2": 100,
    "TU3_2_2_2_1_1": 50,
    "TU3_2_2_2_1_2": 50,
    "TU3_2_2_2_2_1": 100,
    "TU3_2_2_2_2_2": 100,
    "TU3_2_2_3_1_1": 50,
    "TU3_2_2_3_1_2": 50,
    "TU3_2_2_3_2_1": 100,
    "TU3_2_2_3_2_2": 100,
    "TU3_3_1_1_1_1": 15.71,
    "TU3_3_1_1_1_2": 15.71,
    "TU3_3_1_1_2_1": 31.42,
    "TU3_3_1_1_2_2": 31.42,
    "TU3_3_1_2_1_1": 15.71,
    "TU3_3_1_2_1_2": 15.71,
    "TU3_3_1_2_2_1": 31.42,
    "TU3_3_1_2_2_2": 31.42,
    "TU3_3_1_3_1_1": 15.71,
    "TU3_3_1_3_1_2": 15.71,
    "TU3_3_1_3_2_1": 31.42,
    "TU3_3_1_3_2_2": 31.42,
    "TU3_3_2_1_1_1": 50,
    "TU3_3_2_1_1_2": 50,
    "TU3_3_2_1_2_1": 100,
    "TU3_3_2_1_2_2": 100,
    "TU3_3_2_2_1_1": 50,
    "TU3_3_2_2_1_2": 50,
    "TU3_3_2_2_2_1": 100,
    "TU3_3_2_2_2_2": 100,
    "TU3_3_2_3_1_1": 50,
    "TU3_3_2_3_1_2": 50,
    "TU3_3_2_3_2_1": 100,
    "TU3_3_2_3_2_2": 100,    
}

# Dictionary to look up what configuration was used in an experimnet. key: experiment id, value: configuration
TU3_configuration_dict = {
    "TU3_1_1_1_1_1": 1,
    "TU3_1_1_1_1_2": 2,
    "TU3_1_1_1_2_1": 3,
    "TU3_1_1_1_2_2": 4,
    "TU3_1_1_2_1_1": 5,
    "TU3_1_1_2_1_2": 6,
    "TU3_1_1_2_2_1": 7,
    "TU3_1_1_2_2_2": 8,
    "TU3_1_1_3_1_1": 9,
    "TU3_1_1_3_1_2": 10,
    "TU3_1_1_3_2_1": 11,
    "TU3_1_1_3_2_2": 12,
    "TU3_1_2_1_1_1": 13,
    "TU3_1_2_1_1_2": 14,
    "TU3_1_2_1_2_1": 15,
    "TU3_1_2_1_2_2": 16,
    "TU3_1_2_2_1_1": 17,
    "TU3_1_2_2_1_2": 18,
    "TU3_1_2_2_2_1": 19,
    "TU3_1_2_2_2_2": 20,
    "TU3_1_2_3_1_1": 21,
    "TU3_1_2_3_1_2": 22,
    "TU3_1_2_3_2_1": 23,
    "TU3_1_2_3_2_2": 24,
    "TU3_2_1_1_1_1": 1,
    "TU3_2_1_1_1_2": 2,
    "TU3_2_1_1_2_1": 3,
    "TU3_2_1_1_2_2": 4,
    "TU3_2_1_2_1_1": 5,
    "TU3_2_1_2_1_2": 6,
    "TU3_2_1_2_2_1": 7,
    "TU3_2_1_2_2_2": 8,
    "TU3_2_1_3_1_1": 9,
    "TU3_2_1_3_1_2": 10,
    "TU3_2_1_3_2_1": 11,
    "TU3_2_1_3_2_2": 12,
    "TU3_2_2_1_1_1": 13,
    "TU3_2_2_1_1_2": 14,
    "TU3_2_2_1_2_1": 15,
    "TU3_2_2_1_2_2": 16,
    "TU3_2_2_2_1_1": 17,
    "TU3_2_2_2_1_2": 18,
    "TU3_2_2_2_2_1": 19,
    "TU3_2_2_2_2_2": 20,
    "TU3_2_2_3_1_1": 21,
    "TU3_2_2_3_1_2": 22,
    "TU3_2_2_3_2_1": 23,
    "TU3_2_2_3_2_2": 24,
    "TU3_3_1_1_1_1": 1,
    "TU3_3_1_1_1_2": 2,
    "TU3_3_1_1_2_1": 3,
    "TU3_3_1_1_2_2": 4,
    "TU3_3_1_2_1_1": 5,
    "TU3_3_1_2_1_2": 6,
    "TU3_3_1_2_2_1": 7,
    "TU3_3_1_2_2_2": 8,
    "TU3_3_1_3_1_1": 9,
    "TU3_3_1_3_1_2": 10,
    "TU3_3_1_3_2_1": 11,
    "TU3_3_1_3_2_2": 12,
    "TU3_3_2_1_1_1": 13,
    "TU3_3_2_1_1_2": 14,
    "TU3_3_2_1_2_1": 15,
    "TU3_3_2_1_2_2": 16,
    "TU3_3_2_2_1_1": 17,
    "TU3_3_2_2_1_2": 18,
    "TU3_3_2_2_2_1": 19,
    "TU3_3_2_2_2_2": 20,
    "TU3_3_2_3_1_1": 21,
    "TU3_3_2_3_1_2": 22,
    "TU3_3_2_3_2_1": 23,
    "TU3_3_2_3_2_2": 24,    
}

# Dictionary to look up who we are selling the ticket to. key: experiment id, value: buyer
TU3_buyer_dict = {
    "TU3_1_1_1_1_1": "friend",
    "TU3_1_1_1_1_2": "stranger",
    "TU3_1_1_1_2_1": "friend",
    "TU3_1_1_1_2_2": "stranger",
    "TU3_1_1_2_1_1": "friend",
    "TU3_1_1_2_1_2": "stranger",
    "TU3_1_1_2_2_1": "friend",
    "TU3_1_1_2_2_2": "stranger",
    "TU3_1_1_3_1_1": "friend",
    "TU3_1_1_3_1_2": "stranger",
    "TU3_1_1_3_2_1": "friend",
    "TU3_1_1_3_2_2": "stranger",
    "TU3_1_2_1_1_1": "friend",
    "TU3_1_2_1_1_2": "stranger",
    "TU3_1_2_1_2_1": "friend",
    "TU3_1_2_1_2_2": "stranger",
    "TU3_1_2_2_1_1": "friend",
    "TU3_1_2_2_1_2": "stranger",
    "TU3_1_2_2_2_1": "friend",
    "TU3_1_2_2_2_2": "stranger",
    "TU3_1_2_3_1_1": "friend",
    "TU3_1_2_3_1_2": "stranger",
    "TU3_1_2_3_2_1": "friend",
    "TU3_1_2_3_2_2": "stranger",
    "TU3_2_1_1_1_1": "friend",
    "TU3_2_1_1_1_2": "stranger",
    "TU3_2_1_1_2_1": "friend",
    "TU3_2_1_1_2_2": "stranger",
    "TU3_2_1_2_1_1": "friend",
    "TU3_2_1_2_1_2": "stranger",
    "TU3_2_1_2_2_1": "friend",
    "TU3_2_1_2_2_2": "stranger",
    "TU3_2_1_3_1_1": "friend",
    "TU3_2_1_3_1_2": "stranger",
    "TU3_2_1_3_2_1": "friend",
    "TU3_2_1_3_2_2": "stranger",
    "TU3_2_2_1_1_1": "friend",
    "TU3_2_2_1_1_2": "stranger",
    "TU3_2_2_1_2_1": "friend",
    "TU3_2_2_1_2_2": "stranger",
    "TU3_2_2_2_1_1": "friend",
    "TU3_2_2_2_1_2": "stranger",
    "TU3_2_2_2_2_1": "friend",
    "TU3_2_2_2_2_2": "stranger",
    "TU3_2_2_3_1_1": "friend",
    "TU3_2_2_3_1_2": "stranger",
    "TU3_2_2_3_2_1": "friend",
    "TU3_2_2_3_2_2": "stranger",
    "TU3_3_1_1_1_1": "friend",
    "TU3_3_1_1_1_2": "stranger",
    "TU3_3_1_1_2_1": "friend",
    "TU3_3_1_1_2_2": "stranger",
    "TU3_3_1_2_1_1": "friend",
    "TU3_3_1_2_1_2": "stranger",
    "TU3_3_1_2_2_1": "friend",
    "TU3_3_1_2_2_2": "stranger",
    "TU3_3_1_3_1_1": "friend",
    "TU3_3_1_3_1_2": "stranger",
    "TU3_3_1_3_2_1": "friend",
    "TU3_3_1_3_2_2": "stranger",
    "TU3_3_2_1_1_1": "friend",
    "TU3_3_2_1_1_2": "stranger",
    "TU3_3_2_1_2_1": "friend",
    "TU3_3_2_1_2_2": "stranger",
    "TU3_3_2_2_1_1": "friend",
    "TU3_3_2_2_1_2": "stranger",
    "TU3_3_2_2_2_1": "friend",
    "TU3_3_2_2_2_2": "stranger",
    "TU3_3_2_3_1_1": "friend",
    "TU3_3_2_3_1_2": "stranger",
    "TU3_3_2_3_2_1": "friend",
    "TU3_3_2_3_2_2": "stranger",    
}

In [7]:
# Collect dicionairies and save for use in dashboard
TU3_dictionaries = [TU3_experiment_prompts_dict, TU3_model_dict, TU3_prompt_ids_dict, TU3_actual_price_dict,
                     TU3_initial_costs_dict, TU3_orientation_price_dict, TU3_configuration_dict, TU3_buyer_dict]
#with open('Dashboard/src/data/Input/TU3_dictionaries.pkl', 'wb') as f:
 #   pickle.dump(TU3_dictionaries, f)

---

#### Setting up functions to repeatedly prompt the LLMs

- Helper function to extract dollar amount of given answers

In [8]:
def extract_dollar_amounts(answers):
    # Only return values that start with "$"
    valid_prices = [item for item in answers if item.startswith("$") and item[1:].replace(',', '').replace('.', '').isdigit()] # check if everything after $ is a digit, exlcuding commas
    # Delete the "$" from the beginning of each price
    prices = [item.replace('$', '') for item in valid_prices]
    return prices

- Functions to query 1 prompt n times for OpenAI Models

In [10]:
def TU3_run_experiment(experiment_id, n, progress_bar, temperature):
    
    answers = []
    for _ in range(n): 
        response = client.chat.completions.create(
            model = TU3_model_dict[experiment_id], 
            max_tokens = 2,
            temperature = temperature, # range is 0 to 2
            messages = [
            {"role": "system", "content": "Answer by only giving a single price in dollars and cents without an explanation."},        
            {"role": "user", "content": 
             f"{TU3_experiment_prompts_dict[experiment_id]} Answer by only giving a single price in dollars and cents without an explanation."}
                   ])

        # Store the answer in the list
        answer = response.choices[0].message.content
        answers.append(answer.strip())
        # Update progress bar (given from either temperature loop, or set locally)
        progress_bar.update(1)

    # Extract valid prices from answers
    valid_prices = extract_dollar_amounts(answers)

    # Compute number of valid answers
    n_observations = len(valid_prices)

    # Collect results 
    results = [experiment_id, temperature, TU3_model_dict[experiment_id], TU3_actual_price_dict[experiment_id], TU3_initial_costs_dict[experiment_id],
                TU3_orientation_price_dict[experiment_id], TU3_configuration_dict[experiment_id], n_observations, valid_prices, TU3_buyer_dict[experiment_id]]
    

    # Give out results
    return results

- Adjusted function for Dashboard (returns dataframe right away)

In [11]:
def TU3_run_experiment_dashboard(experiment_id, n, temperature):
    
    answers = []
    for _ in range(n): 
        response = client.chat.completions.create(
            model = TU3_model_dict[experiment_id], 
            max_tokens = 2,
            temperature = temperature, # range is 0 to 2
            messages = [
            {"role": "system", "content": "Answer by only giving a single price in dollars and cents without an explanation."},        
            {"role": "user", "content": 
             f"{TU3_experiment_prompts_dict[experiment_id]} Answer by only giving a single price in dollars and cents without an explanation."}
                   ])

        # Store the answer in the list
        answer = response.choices[0].message.content
        answers.append(answer.strip())

    # Extract valid prices from answers
    valid_prices = extract_dollar_amounts(answers)

    # Compute number of valid answers
    n_observations = len(valid_prices)

    # Collect results 
    results = pd.DataFrame([experiment_id, temperature, TU3_model_dict[experiment_id], TU3_actual_price_dict[experiment_id], TU3_initial_costs_dict[experiment_id],
                TU3_orientation_price_dict[experiment_id], TU3_configuration_dict[experiment_id], n_observations, f"{valid_prices}", TU3_buyer_dict[experiment_id]])
    results = results.set_index(pd.Index(["Experiment_id", "Temperature", "Model", "Actual_price", "Initial_cost", "Orientation_price", "Configuration", "Obs.", "Answers", "Buyer"]))
    results = results.transpose()
    

    # Give out results
    return results

- Functions to query 1 prompt n times (LLama)

In [13]:
def TU3_run_experiment_llama(experiment_id, n, progress_bar, temperature):
    answers = []
    for _ in range(n):
        response = replicate.run(
            TU3_model_dict[experiment_id],
            input = {
                "system_prompt":  "Answer by only giving a single price in dollars and cents without an explanation.",
                "temperature": temperature,
                "max_new_tokens": 10, 
                "prompt": f"{TU3_experiment_prompts_dict[experiment_id]} Answer by only giving a single price in dollars and cents without an explanation."
            }
        )
        # Grab answer and append to list
        answer = "" # Set to empty string, otherwise it would append the previous answer to the new one
        for item in response:
            answer = answer + item
        answers.append(answer.strip())

        # Update progress bar
        progress_bar.update(1)

    
    # Extract valid prices from answers
    valid_prices = extract_dollar_amounts(answers)

    # Compute number of valid answers
    n_observations = len(valid_prices)

    # Collect results 
    results = [experiment_id, temperature, TU3_model_dict[experiment_id], TU3_actual_price_dict[experiment_id], TU3_initial_costs_dict[experiment_id],
                TU3_orientation_price_dict[experiment_id], TU3_configuration_dict[experiment_id], n_observations, valid_prices, TU3_buyer_dict[experiment_id]]

    # Give out results
    return results

- Adjusted function for Dashboard (returns dataframe right away)

In [14]:
def TU3_run_experiment_llama_dashboard(experiment_id, n, temperature):
    answers = []
    for _ in range(n):
        response = replicate.run(
            TU3_model_dict[experiment_id],
            input = {
                "system_prompt":  "Answer by only giving a single price in dollars and cents without an explanation.",
                "temperature": temperature,
                "max_new_tokens": 10, 
                "prompt": f"{TU3_experiment_prompts_dict[experiment_id]} Answer by only giving a single price in dollars and cents without an explanation."
            }
        )
        # Grab answer and append to list
        answer = "" # Set to empty string, otherwise it would append the previous answer to the new one
        for item in response:
            answer = answer + item
        answers.append(answer.strip())
   
    # Extract valid prices from answers
    valid_prices = extract_dollar_amounts(answers)

    # Compute number of valid answers
    n_observations = len(valid_prices)

    # Collect results 
    results = pd.DataFrame([experiment_id, temperature, TU3_model_dict[experiment_id], TU3_actual_price_dict[experiment_id], TU3_initial_costs_dict[experiment_id],
                TU3_orientation_price_dict[experiment_id], TU3_configuration_dict[experiment_id], n_observations, f"{valid_prices}", TU3_buyer_dict[experiment_id]])
    results = results.set_index(pd.Index(["Experiment_id", "Temperature", "Model", "Actual_price", "Initial_cost", "Orientation_price", "Configuration", "Obs.", "Answers", "Buyer"]))
    results = results.transpose()
    

    # Give out results
    return results

- Function to loop run_experiment() over a list of temperature values

In [15]:
def TU3_temperature_loop(function, experiment_id, temperature_list = [0.01, 0.5, 1, 1.5, 2], n = 50):
    """
    Function to run an experiment with different temperature values.
    
    Args:
        function (function): Function to be used for querying ChatGPT i.e. run_experiment()
        experiment_id (str): ID of th e experiment to be run. Contains info about prompt and model
        temperature_list (list): List of temperature values to be looped over
        n: Number of requests for each prompt per temperature value
        max_tokens: Maximum number of tokens in response object
        
    Returns:
        results_df: Dataframe with experiment results
        probs_df: Dataframe with answer probabilities
    """    
    # Empty list for storing results
    results_list = []

    # Initialize progress bar -> used as input for run_experiment()
    progress_bar = tqdm(range(n*len(temperature_list)))

    # Loop over different temperature values, calling the input function n times each (i.e. queriyng ChatGPT n times)
    for temperature in temperature_list:
        results = function(experiment_id = experiment_id, n = n, temperature = temperature, progress_bar = progress_bar) 
        results_list.append(results)
       

    # Horizontally concatenate the results, transpose, and set index
    results_df = pd.DataFrame(results_list).transpose().set_index(pd.Index(
        ["Experiment_id", "Temperature", "Model", "Actual_price", "Initial_cost", "Orientation_price", "Configuration", "Obs.", "Answers", "Buyer"])).transpose()
  
   
    # Return some information about the experiment as a check
    check = f"In this run, a total of {n*len(temperature_list)} requests were made using {TU3_prompt_ids_dict[experiment_id]}."
    # Print information about the experiment
    print(check)
 
    return results_df

---

### Model 1: GPT-3.5-Turbo

In [16]:
# Set number of requests per temperature value
N = 2

- Configuration 1

In [17]:
results_1 = []
results_1_1 = TU3_temperature_loop(TU3_run_experiment, "TU3_1_1_1_1_1", n = N)
results_1.append(results_1_1)   

  0%|          | 0/10 [00:00<?, ?it/s]

100%|██████████| 10/10 [00:05<00:00,  1.95it/s]

In this run, a total of 10 requests were made using TU3_prompts[0].


- Configuration 2

In [18]:
results_2 = []
results_2_1 = TU3_temperature_loop(TU3_run_experiment, "TU3_1_1_1_1_2", n = N)
results_2.append(results_2_1)

100%|██████████| 10/10 [00:04<00:00,  2.46it/s]

In this run, a total of 10 requests were made using TU3_prompts[1].


- Configuration 3

In [19]:
results_3 = []
results_3_1 = TU3_temperature_loop(TU3_run_experiment, "TU3_1_1_1_2_1", n = N)
results_3.append(results_3_1)

100%|██████████| 10/10 [00:05<00:00,  1.88it/s]

In this run, a total of 10 requests were made using TU3_prompts[2].


- Configuration 4


In [20]:
results_4 = []
results_4_1 = TU3_temperature_loop(TU3_run_experiment, "TU3_1_1_1_2_2", n = N)
results_4.append(results_4_1)

100%|██████████| 10/10 [00:04<00:00,  2.01it/s]

In this run, a total of 10 requests were made using TU3_prompts[3].


- Configuration 5


In [21]:
results_5 = []
results_5_1 = TU3_temperature_loop(TU3_run_experiment, "TU3_1_1_2_1_1", n = N)
results_5.append(results_5_1)


100%|██████████| 10/10 [00:04<00:00,  2.11it/s]

In this run, a total of 10 requests were made using TU3_prompts[4].


- Configuration 6


In [22]:
results_6 = []
results_6_1 = TU3_temperature_loop(TU3_run_experiment, "TU3_1_1_2_1_2", n = N)
results_6.append(results_6_1)

100%|██████████| 10/10 [00:04<00:00,  2.18it/s]

In this run, a total of 10 requests were made using TU3_prompts[5].


- Configuration 7


In [23]:
results_7 = []
results_7_1 = TU3_temperature_loop(TU3_run_experiment, "TU3_1_1_2_2_1", n = N)
results_7.append(results_7_1)

100%|██████████| 10/10 [00:04<00:00,  2.13it/s]

In this run, a total of 10 requests were made using TU3_prompts[6].


- Configuration 8

In [24]:
results_8 = []
results_8_1 = TU3_temperature_loop(TU3_run_experiment, "TU3_1_1_2_2_2", n = N)
results_8.append(results_8_1)

100%|██████████| 10/10 [00:04<00:00,  2.22it/s]

In this run, a total of 10 requests were made using TU3_prompts[7].


- Configuration 9

In [25]:
results_9 = []
results_9_1 = TU3_temperature_loop(TU3_run_experiment, "TU3_1_1_3_1_1", n = N)
results_9.append(results_9_1)

100%|██████████| 10/10 [00:05<00:00,  1.74it/s]

In this run, a total of 10 requests were made using TU3_prompts[8].


- Configuration 10

In [26]:
results_10 = []
results_10_1 = TU3_temperature_loop(TU3_run_experiment, "TU3_1_1_3_1_2", n = N)
results_10.append(results_10_1)

100%|██████████| 10/10 [00:03<00:00,  2.59it/s]

In this run, a total of 10 requests were made using TU3_prompts[9].


- Configuration 11

In [27]:
results_11 = []
results_11_1 = TU3_temperature_loop(TU3_run_experiment, "TU3_1_1_3_2_1", n = N)
results_11.append(results_11_1)

100%|██████████| 10/10 [00:08<00:00,  1.24it/s]

In this run, a total of 10 requests were made using TU3_prompts[10].


- Configuration 12

In [28]:
results_12 = []
results_12_1 = TU3_temperature_loop(TU3_run_experiment, "TU3_1_1_3_2_2", n = N)
results_12.append(results_12_1)

100%|██████████| 10/10 [00:07<00:00,  1.31it/s]

In this run, a total of 10 requests were made using TU3_prompts[11].


- Configuration 13

In [29]:
results_13 = []
results_13_1 = TU3_temperature_loop(TU3_run_experiment, "TU3_1_2_1_1_1", n = N)
results_13.append(results_13_1)

100%|██████████| 10/10 [00:05<00:00,  1.99it/s]

In this run, a total of 10 requests were made using TU3_prompts[12].


- Configuration 14

In [30]:
results_14 = []
results_14_1 = TU3_temperature_loop(TU3_run_experiment, "TU3_1_2_1_1_2", n = N)
results_14.append(results_14_1)

100%|██████████| 10/10 [00:04<00:00,  2.19it/s]

In this run, a total of 10 requests were made using TU3_prompts[13].


- Configuration 15

In [31]:
results_15 = []
results_15_1 = TU3_temperature_loop(TU3_run_experiment, "TU3_1_2_1_2_1", n = N)
results_15.append(results_15_1)

100%|██████████| 10/10 [00:04<00:00,  2.14it/s]

In this run, a total of 10 requests were made using TU3_prompts[14].


- Configuration 16

In [32]:
results_16 = []
results_16_1 = TU3_temperature_loop(TU3_run_experiment, "TU3_1_2_1_2_2", n = N)
results_16.append(results_16_1)

100%|██████████| 10/10 [00:03<00:00,  2.92it/s]

In this run, a total of 10 requests were made using TU3_prompts[15].


- Configuration 17

In [33]:
results_17 = []
results_17_1 = TU3_temperature_loop(TU3_run_experiment, "TU3_1_2_2_1_1", n = N)
results_17.append(results_17_1)

100%|██████████| 10/10 [00:06<00:00,  1.58it/s]

In this run, a total of 10 requests were made using TU3_prompts[16].


- Configuration 18

In [34]:
results_18 = []
results_18_1 = TU3_temperature_loop(TU3_run_experiment, "TU3_1_2_2_1_2", n = N)
results_18.append(results_18_1)

100%|██████████| 10/10 [00:05<00:00,  1.86it/s]

In this run, a total of 10 requests were made using TU3_prompts[17].


- Configuration 19

In [35]:
results_19 = []
results_19_1 = TU3_temperature_loop(TU3_run_experiment, "TU3_1_2_2_2_1", n = N)
results_19.append(results_19_1)

100%|██████████| 10/10 [00:04<00:00,  2.33it/s]

In this run, a total of 10 requests were made using TU3_prompts[18].


- Configuration 20

In [36]:
results_20 = []
results_20_1 = TU3_temperature_loop(TU3_run_experiment, "TU3_1_2_2_2_2", n = N)
results_20.append(results_20_1)

100%|██████████| 10/10 [00:04<00:00,  2.29it/s]

In this run, a total of 10 requests were made using TU3_prompts[19].


- Configuration 21

In [37]:
results_21 = []
results_21_1 = TU3_temperature_loop(TU3_run_experiment, "TU3_1_2_3_1_1", n = N)
results_21.append(results_21_1)

100%|██████████| 10/10 [00:05<00:00,  1.83it/s]

In this run, a total of 10 requests were made using TU3_prompts[20].


- Configuration 22

In [38]:
results_22 = []
results_22_1 = TU3_temperature_loop(TU3_run_experiment, "TU3_1_2_3_1_2", n = N)
results_22.append(results_22_1)

100%|██████████| 10/10 [00:04<00:00,  2.03it/s]

In this run, a total of 10 requests were made using TU3_prompts[21].


- Configuration 23

In [39]:
results_23 = []
results_23_1 = TU3_temperature_loop(TU3_run_experiment, "TU3_1_2_3_2_1", n = N)
results_23.append(results_23_1)

100%|██████████| 10/10 [00:04<00:00,  2.33it/s]

In this run, a total of 10 requests were made using TU3_prompts[22].


- Configuration 24

In [40]:
results_24 = []
results_24_1 = TU3_temperature_loop(TU3_run_experiment, "TU3_1_2_3_2_2", n = N)
results_24.append(results_24_1)

100%|██████████| 10/10 [00:05<00:00,  1.99it/s]

In this run, a total of 10 requests were made using TU3_prompts[23].


In [41]:
TU3_gpt35_results = [results_1_1, results_2_1, results_3_1, results_4_1, results_5_1, results_6_1, results_7_1, results_8_1, results_9_1, results_10_1, results_11_1, results_12_1,
                      results_13_1, results_14_1, results_15_1, results_16_1, results_17_1, results_18_1, results_19_1, results_20_1, results_21_1, results_22_1, results_23_1, results_24_1]
TU3_gpt35_results_df = pd.concat(TU3_gpt35_results)

---

### Model 2: GPT-4-1106-Preview

In [42]:
# Select number of requests per temperature value
N = 2

- Configuration 1

In [43]:
results_1_2 = TU3_temperature_loop(TU3_run_experiment, "TU3_2_1_1_1_1", n = N)
results_1.append(results_1_2)

100%|██████████| 10/10 [00:07<00:00,  1.31it/s]

In this run, a total of 10 requests were made using TU3_prompts[0].


- Configuration 2

In [44]:
results_2_2 = TU3_temperature_loop(TU3_run_experiment, "TU3_2_1_1_1_2", n = N)
results_2.append(results_2_2)

100%|██████████| 10/10 [00:09<00:00,  1.07it/s]

In this run, a total of 10 requests were made using TU3_prompts[1].


- Configuration 3

In [45]:
results_3_2 = TU3_temperature_loop(TU3_run_experiment, "TU3_2_1_1_2_1", n = N)
results_3.append(results_3_2)

100%|██████████| 10/10 [00:10<00:00,  1.04s/it]

In this run, a total of 10 requests were made using TU3_prompts[2].


- Configuration 4

In [46]:
results_4_2 = TU3_temperature_loop(TU3_run_experiment, "TU3_2_1_1_2_2", n = N)
results_4.append(results_4_2)

100%|██████████| 10/10 [00:10<00:00,  1.05s/it]

In this run, a total of 10 requests were made using TU3_prompts[3].


- Configuration 5

In [47]:
results_5_2 = TU3_temperature_loop(TU3_run_experiment, "TU3_2_1_2_1_1", n = N)
results_5.append(results_5_2)

100%|██████████| 10/10 [00:09<00:00,  1.11it/s]

In this run, a total of 10 requests were made using TU3_prompts[4].


- Configuration 6

In [48]:
results_6_2 = TU3_temperature_loop(TU3_run_experiment, "TU3_2_1_2_1_2", n = N)
results_6.append(results_6_2)

100%|██████████| 10/10 [00:06<00:00,  1.58it/s]

In this run, a total of 10 requests were made using TU3_prompts[5].


- Configuration 7

In [49]:
results_7_2 = TU3_temperature_loop(TU3_run_experiment, "TU3_2_1_2_2_1", n = N)
results_7.append(results_7_2)

100%|██████████| 10/10 [00:08<00:00,  1.17it/s]

In this run, a total of 10 requests were made using TU3_prompts[6].


- Configuration 8

In [50]:
results_8_2 = TU3_temperature_loop(TU3_run_experiment, "TU3_2_1_2_2_2", n = N)
results_8.append(results_8_2)

100%|██████████| 10/10 [00:07<00:00,  1.30it/s]

In this run, a total of 10 requests were made using TU3_prompts[7].


- Configuration 9

In [51]:
results_9_2 = TU3_temperature_loop(TU3_run_experiment, "TU3_2_1_3_1_1", n = N)
results_9.append(results_9_2)

100%|██████████| 10/10 [00:26<00:00,  2.67s/it]

In this run, a total of 10 requests were made using TU3_prompts[8].


- Configuration 10

In [52]:
results_10_2 = TU3_temperature_loop(TU3_run_experiment, "TU3_2_1_3_1_2", n = N)
results_10.append(results_10_2)

100%|██████████| 10/10 [00:10<00:00,  1.01s/it]

In this run, a total of 10 requests were made using TU3_prompts[9].


- Configuration 11

In [53]:
results_11_2 = TU3_temperature_loop(TU3_run_experiment, "TU3_2_1_3_2_1", n = N)
results_11.append(results_11_2)

100%|██████████| 10/10 [00:09<00:00,  1.08it/s]

In this run, a total of 10 requests were made using TU3_prompts[10].


- Configuration 12

In [54]:
results_12_2 = TU3_temperature_loop(TU3_run_experiment, "TU3_2_1_3_2_2", n = N)
results_12.append(results_12_2)

100%|██████████| 10/10 [00:08<00:00,  1.24it/s]

In this run, a total of 10 requests were made using TU3_prompts[11].


- Configuration 13

In [55]:
results_13_2 = TU3_temperature_loop(TU3_run_experiment, "TU3_2_2_1_1_1", n = N)
results_13.append(results_13_2)

100%|██████████| 10/10 [00:07<00:00,  1.33it/s]

In this run, a total of 10 requests were made using TU3_prompts[12].


- Configuration 14

In [56]:
results_14_2 = TU3_temperature_loop(TU3_run_experiment, "TU3_2_2_1_1_2", n = N)
results_14.append(results_14_2)

100%|██████████| 10/10 [00:07<00:00,  1.28it/s]

In this run, a total of 10 requests were made using TU3_prompts[13].


- Configuration 15

In [57]:
results_15_2 = TU3_temperature_loop(TU3_run_experiment, "TU3_2_2_1_2_1", n = N)
results_15.append(results_15_2)

100%|██████████| 10/10 [00:12<00:00,  1.24s/it]

In this run, a total of 10 requests were made using TU3_prompts[14].


- Configuration 16

In [58]:
results_16_2 = TU3_temperature_loop(TU3_run_experiment, "TU3_2_2_1_2_2", n = N)
results_16.append(results_16_2)

100%|██████████| 10/10 [00:12<00:00,  1.30s/it]

In this run, a total of 10 requests were made using TU3_prompts[15].


- Configuration 17

In [59]:
results_17_2 = TU3_temperature_loop(TU3_run_experiment, "TU3_2_2_2_1_1", n = N)
results_17.append(results_17_2)

100%|██████████| 10/10 [00:09<00:00,  1.01it/s]

In this run, a total of 10 requests were made using TU3_prompts[16].


- Configuration 18

In [60]:
results_18_2 = TU3_temperature_loop(TU3_run_experiment, "TU3_2_2_2_1_2", n = N)
results_18.append(results_18_2)

100%|██████████| 10/10 [00:06<00:00,  1.55it/s]

In this run, a total of 10 requests were made using TU3_prompts[17].


- Configuration 19

In [61]:
results_19_2 = TU3_temperature_loop(TU3_run_experiment, "TU3_2_2_2_2_1", n = N)
results_19.append(results_19_2)

100%|██████████| 10/10 [00:10<00:00,  1.01s/it]

In this run, a total of 10 requests were made using TU3_prompts[18].


- Configuration 20

In [62]:
results_20_2 = TU3_temperature_loop(TU3_run_experiment, "TU3_2_2_2_2_2", n = N)
results_20.append(results_20_2)

100%|██████████| 10/10 [00:09<00:00,  1.03it/s]

In this run, a total of 10 requests were made using TU3_prompts[19].


- Configuration 21

In [63]:
results_21_2 = TU3_temperature_loop(TU3_run_experiment, "TU3_2_2_3_1_1", n = N)
results_21.append(results_21_2)

100%|██████████| 10/10 [00:07<00:00,  1.33it/s]

In this run, a total of 10 requests were made using TU3_prompts[20].


- Configuration 22

In [64]:
results_22_2 = TU3_temperature_loop(TU3_run_experiment, "TU3_2_2_3_1_2", n = N)
results_22.append(results_22_2)

100%|██████████| 10/10 [00:09<00:00,  1.05it/s]

In this run, a total of 10 requests were made using TU3_prompts[21].


- Configuration 23

In [65]:
results_23_2 = TU3_temperature_loop(TU3_run_experiment, "TU3_2_2_3_2_1", n = N)
results_23.append(results_23_2)

100%|██████████| 10/10 [00:08<00:00,  1.13it/s]

In this run, a total of 10 requests were made using TU3_prompts[22].


- Configuration 24

In [66]:
results_24_2 = TU3_temperature_loop(TU3_run_experiment, "TU3_2_2_3_2_2", n = N)
results_24.append(results_24_2)

100%|██████████| 10/10 [00:06<00:00,  1.52it/s]

In this run, a total of 10 requests were made using TU3_prompts[23].


In [67]:
TU3_gpt4_results = [results_1_2, results_2_2, results_3_2, results_4_2, results_5_2, results_6_2, results_7_2, results_8_2, results_9_2, results_10_2, results_11_2, results_12_2,
                        results_13_2, results_14_2, results_15_2, results_16_2, results_17_2, results_18_2, results_19_2, results_20_2, results_21_2, results_22_2, results_23_2, results_24_2]
TU3_gpt4_results_df = pd.concat(TU3_gpt4_results)

---

### Model 3: LLama-2-70b

In [68]:
# Select number of requests per temperature value
N = 2

- Configuration 1

In [69]:
results_1_3 = TU3_temperature_loop(TU3_run_experiment_llama, "TU3_3_1_1_1_1", n = N)

100%|██████████| 10/10 [00:32<00:00,  3.21s/it]

In this run, a total of 10 requests were made using TU3_prompts[0].


- Configuration 2

In [70]:
results_2_3 = TU3_temperature_loop(TU3_run_experiment_llama, "TU3_3_1_1_1_2", n = N)

100%|██████████| 10/10 [00:18<00:00,  1.81s/it]

In this run, a total of 10 requests were made using TU3_prompts[1].


- Configuration 3

In [71]:
results_3_3 = TU3_temperature_loop(TU3_run_experiment_llama, "TU3_3_1_1_2_1", n = N)

100%|██████████| 10/10 [00:18<00:00,  1.84s/it]

In this run, a total of 10 requests were made using TU3_prompts[2].


- Configuration 4

In [72]:
results_4_3 = TU3_temperature_loop(TU3_run_experiment_llama, "TU3_3_1_1_2_2", n = N)

100%|██████████| 10/10 [00:21<00:00,  2.13s/it]

In this run, a total of 10 requests were made using TU3_prompts[3].


- Configuration 5

In [73]:
results_5_3 = TU3_temperature_loop(TU3_run_experiment_llama, "TU3_3_1_2_1_1", n = N)

100%|██████████| 10/10 [00:18<00:00,  1.85s/it]

In this run, a total of 10 requests were made using TU3_prompts[4].


- Configuration 6

In [74]:
results_6_3 = TU3_temperature_loop(TU3_run_experiment_llama, "TU3_3_1_2_1_2", n = N)

100%|██████████| 10/10 [00:18<00:00,  1.84s/it]

In this run, a total of 10 requests were made using TU3_prompts[5].


- Configuration 7

In [75]:
results_7_3 = TU3_temperature_loop(TU3_run_experiment_llama, "TU3_3_1_2_2_1", n = N)

100%|██████████| 10/10 [00:22<00:00,  2.22s/it]

In this run, a total of 10 requests were made using TU3_prompts[6].


- Configuration 8

In [76]:
results_8_3 = TU3_temperature_loop(TU3_run_experiment_llama, "TU3_3_1_2_2_2", n = N)

100%|██████████| 10/10 [00:18<00:00,  1.82s/it]

In this run, a total of 10 requests were made using TU3_prompts[7].


- Configuration 9

In [77]:
results_9_3 = TU3_temperature_loop(TU3_run_experiment_llama, "TU3_3_1_3_1_1", n = N)

100%|██████████| 10/10 [00:34<00:00,  3.45s/it]

In this run, a total of 10 requests were made using TU3_prompts[8].


- Configuration 10

In [78]:
results_10_3 = TU3_temperature_loop(TU3_run_experiment_llama, "TU3_3_1_3_1_2", n = N)

100%|██████████| 10/10 [00:18<00:00,  1.83s/it]

In this run, a total of 10 requests were made using TU3_prompts[9].


- Configuration 11

In [79]:
results_11_3 = TU3_temperature_loop(TU3_run_experiment_llama, "TU3_3_1_3_2_1", n = N)

100%|██████████| 10/10 [00:18<00:00,  1.83s/it]

In this run, a total of 10 requests were made using TU3_prompts[10].


- Configuration 12

In [80]:
results_12_3 = TU3_temperature_loop(TU3_run_experiment_llama, "TU3_3_1_3_2_2", n = N)

100%|██████████| 10/10 [00:20<00:00,  2.10s/it]

In this run, a total of 10 requests were made using TU3_prompts[11].


- Configuration 13

In [81]:
results_13_3 = TU3_temperature_loop(TU3_run_experiment_llama, "TU3_3_2_1_1_1", n = N)

100%|██████████| 10/10 [00:11<00:00,  1.12s/it]

In this run, a total of 10 requests were made using TU3_prompts[12].


- Configuration 14

In [82]:
results_14_3 = TU3_temperature_loop(TU3_run_experiment_llama, "TU3_3_2_1_1_2", n = N)

100%|██████████| 10/10 [00:24<00:00,  2.46s/it]

In this run, a total of 10 requests were made using TU3_prompts[13].


- Configuration 15

In [83]:
results_15_3 = TU3_temperature_loop(TU3_run_experiment_llama, "TU3_3_2_1_2_1", n = N)

100%|██████████| 10/10 [00:11<00:00,  1.10s/it]

In this run, a total of 10 requests were made using TU3_prompts[14].


- Configuration 16

In [84]:
results_16_3 = TU3_temperature_loop(TU3_run_experiment_llama, "TU3_3_2_1_2_2", n = N)

100%|██████████| 10/10 [00:26<00:00,  2.60s/it]

In this run, a total of 10 requests were made using TU3_prompts[15].


- Configuration 17

In [85]:
results_17_3 = TU3_temperature_loop(TU3_run_experiment_llama, "TU3_3_2_2_1_1", n = N)

100%|██████████| 10/10 [00:16<00:00,  1.61s/it]

In this run, a total of 10 requests were made using TU3_prompts[16].


- Configuration 18

In [86]:
results_18_3 = TU3_temperature_loop(TU3_run_experiment_llama, "TU3_3_2_2_1_2", n = N)

100%|██████████| 10/10 [00:11<00:00,  1.10s/it]

In this run, a total of 10 requests were made using TU3_prompts[17].


- Configuration 19

In [87]:
results_19_3 = TU3_temperature_loop(TU3_run_experiment_llama, "TU3_3_2_2_2_1", n = N)

100%|██████████| 10/10 [00:11<00:00,  1.13s/it]

In this run, a total of 10 requests were made using TU3_prompts[18].


- Configuration 20

In [88]:
results_20_3 = TU3_temperature_loop(TU3_run_experiment_llama, "TU3_3_2_2_2_2", n = N)

100%|██████████| 10/10 [00:14<00:00,  1.49s/it]

In this run, a total of 10 requests were made using TU3_prompts[19].


- Configuration 21

In [89]:
results_21_3 = TU3_temperature_loop(TU3_run_experiment_llama, "TU3_3_2_3_1_1", n = N)

100%|██████████| 10/10 [00:11<00:00,  1.12s/it]

In this run, a total of 10 requests were made using TU3_prompts[20].


- Configuration 22

In [90]:
results_22_3 = TU3_temperature_loop(TU3_run_experiment_llama, "TU3_3_2_3_1_2", n = N)

100%|██████████| 10/10 [00:11<00:00,  1.11s/it]

In this run, a total of 10 requests were made using TU3_prompts[21].


- Configuration 23

In [91]:
results_23_3 = TU3_temperature_loop(TU3_run_experiment_llama, "TU3_3_2_3_2_1", n = N)

100%|██████████| 10/10 [00:12<00:00,  1.21s/it]

In this run, a total of 10 requests were made using TU3_prompts[22].


- Configuration 24

In [92]:
results_24_3 = TU3_temperature_loop(TU3_run_experiment_llama, "TU3_3_2_3_2_2", n = N)

100%|██████████| 10/10 [00:12<00:00,  1.26s/it]

In this run, a total of 10 requests were made using TU3_prompts[23].


In [96]:
TU3_llama_results = [results_1_3, results_2_3, results_3_3, results_4_3, results_5_3, results_6_3, results_7_3, results_8_3, results_9_3, results_10_3, results_11_3, results_12_3,
                        results_13_3, results_14_3, results_15_3, results_16_3, results_17_3, results_18_3, results_19_3, results_20_3, results_21_3, results_22_3, results_23_3, results_24_3]
TU3_llama_results_df = pd.concat(TU3_llama_results)

---

#### Collect results and save to .csv

In [106]:
# Combine results
TU3_results = [TU3_gpt35_results_df, TU3_gpt4_results_df, TU3_llama_results_df]
TU3_results_df = pd.concat(TU3_results)


# Rename LLama model
TU3_results_df['Model'] = TU3_results_df['Model'].replace('meta/llama-2-70b-chat:02e509c789964a7ea8736978a43525956ef40397be9033abf9fd2badfe68c9e3', 
                                  'llama-2-70b')

# Display results
TU3_results_df

,Experiment_id,Temperature,Model,Actual_price,Initial_cost,Orientation_price,Configuration,Obs.,Answers,Buyer
0,TU3_1_1_1_1_1,0.01,gpt-3.5-turbo,15.71,0,15.71,1,2,"[15, 15]",friend
1,TU3_1_1_1_1_1,0.5,gpt-3.5-turbo,15.71,0,15.71,1,2,"[15, 15]",friend
2,TU3_1_1_1_1_1,1.0,gpt-3.5-turbo,15.71,0,15.71,1,2,"[15, 15]",friend
3,TU3_1_1_1_1_1,1.5,gpt-3.5-turbo,15.71,0,15.71,1,1,[0],friend
4,TU3_1_1_1_1_1,2.0,gpt-3.5-turbo,15.71,0,15.71,1,2,"[15, 15]",friend
...,...,...,...,...,...,...,...,...,...,...
0,TU3_3_2_3_2_2,0.01,llama-2-70b,50,100,100,24,2,"[125, 125]",stranger
1,TU3_3_2_3_2_2,0.5,llama-2-70b,50,100,100,24,2,"[125, 125]",stranger
2,TU3_3_2_3_2_2,1.0,llama-2-70b,50,100,100,24,2,"[125, 125]",stranger
3,TU3_3_2_3_2_2,1.5,llama-2-70b,50,100,100,24,2,"[125, 125]",stranger


In [ ]:
# Save to csv
#TU3_results_df.iloc[:,1:].to_csv("Dashboard/src/data/Output/TU3_results_df.csv", index = False)

---

### Visualization of results

In [54]:
TU3_results = pd.read_csv("Dashboard/src/data/Output/TU3_results.csv")
TU3_results

,Experiment_id,Temperature,Model,Actual_price,Initial_cost,Orientation_price,Configuration,Obs.,Answers,Buyer
0,TU3_1_1_1_1_1,0.01,gpt-3.5-turbo,15.71,0.0,15.71,1,100,"['15', '15', '15', '15', '15', '15', '15', '15...",friend
1,TU3_1_1_1_1_1,0.50,gpt-3.5-turbo,15.71,0.0,15.71,1,100,"['15', '15', '15', '15', '15', '15', '15', '15...",friend
2,TU3_1_1_1_1_1,1.00,gpt-3.5-turbo,15.71,0.0,15.71,1,100,"['15', '15', '15', '15', '15', '15', '15', '15...",friend
3,TU3_1_1_1_1_1,1.50,gpt-3.5-turbo,15.71,0.0,15.71,1,91,"['15', '15', '15', '15', '15', '15', '15', '15...",friend
4,TU3_1_1_1_1_1,2.00,gpt-3.5-turbo,15.71,0.0,15.71,1,64,"['15', '15', '15', '15', '15', '0', '16', '5',...",friend
...,...,...,...,...,...,...,...,...,...,...
355,TU3_3_2_3_2_2,0.01,llama-2-70b,50.00,100.0,100.00,24,50,"['125', '125', '125', '125', '125', '125', '12...",stranger
356,TU3_3_2_3_2_2,0.50,llama-2-70b,50.00,100.0,100.00,24,50,"['125', '125', '125', '125', '125', '125', '12...",stranger
357,TU3_3_2_3_2_2,1.00,llama-2-70b,50.00,100.0,100.00,24,50,"['125', '125', '125', '125', '125', '125', '12...",stranger
358,TU3_3_2_3_2_2,1.50,llama-2-70b,50.00,100.0,100.00,24,50,"['125', '125', '125', '125', '125', '125', '12...",stranger


In [73]:
actual_price = 15.71
initial_cost = 15.71
orientation_price = 15.71
buyer = "stranger"
model = "llama-2-70b"
temperature = 2
df = TU3_results[(TU3_results["Actual_price"] == actual_price) & (TU3_results["Initial_cost"] == initial_cost) & (TU3_results["Orientation_price"] == orientation_price)
                  & (TU3_results["Buyer"] == buyer) & (TU3_results["Model"] == model) & (TU3_results["Temperature"] == temperature)]       

In [16]:
def TU3_plot_results(df):

    # Transpose for plotting
    df = df.transpose()
    # Extract model name from the dataframe
    model = df.loc["Model"].iloc[0]
    if model == "meta/llama-2-70b-chat:02e509c789964a7ea8736978a43525956ef40397be9033abf9fd2badfe68c9e3":
        model = "llama-2-70b"

    # Get temperature value 
    temperature = df.loc["Temperature"].iloc[0]
    # Get number of observations 
    n_observations = df.loc["Obs."].iloc[0]
    # Extract model answers
    answers = df.loc["Answers"].apply(literal_eval).iloc[0]
    sorted_answers = sorted(answers, key=lambda x: float(x))
    numeric_answers = [float(answer) for answer in answers]

    # Get number of unique answers
    num_unique_answers = len(set(answers))
    # Get actual ticker price
    actual_price = df.loc["Actual_price"].iloc[0]
    # Get current market price
    current_price = df.loc["Orientation_price"].iloc[0]
    # Get price paid
    initial_cost = df.loc["Initial_cost"].iloc[0]
    # Get buyer 
    buyer = df.loc["Buyer"].iloc[0].capitalize()
    # Compute mean
    mean = np.round(np.mean(numeric_answers),2).astype(str)
    median = np.round(np.median(numeric_answers),2).astype(str)
   

    fig = go.Figure(data = [
    go.Histogram(x = sorted_answers,
                    customdata = [n_observations] * num_unique_answers,
                    hovertemplate = "Price asked: $%{x} <br>Frequency: %{y}<br>Total answers: %{customdata}<br><extra></extra>",

                    marker_color = "rgb(55, 83, 109)",
                    name = f"Actual price: ${actual_price}<br>Initial costs: ${initial_cost}<br>Current price: ${current_price}<br>Buyer: {buyer}<br>Mean: ${mean}<br>Median: ${median}"),
    ])

    # Layout
    fig.update_layout(
        xaxis=dict(
            title="Price asked ($)",
            titlefont_size=18,
            tickfont_size=16,
            tickformat=".2f",
        ),
        yaxis=dict(
            title="Frequency",
            titlefont_size=18,
            tickfont_size=16,
        ),
        title=dict(
            text=f"Distribution of {model}'s answers for temperature {temperature}",
            x=0.45,
            y=0.95,
            font_size=18,
        ),
        legend=dict(
            x=1.01,  
            y=0.9,
            font=dict(family='Arial', size=16, color='black'),
            bordercolor='black',  
            borderwidth=2,           
        ),
        showlegend=True,
        width=1000,
        margin=dict(t=60),
    )
    

    # Show the plot
    return fig

In [18]:
TU3_plot_results(test_results)